In [ ]:
import numpy as np
import heapq
from matplotlib import pyplot as plt

In [ ]:
test_text = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

In [ ]:
with open("input.txt") as f:
    data_text = f.read()

In [ ]:
def add_tuple(a, b):
    return(a[0] + b[0], a[1] + b[1])

def check_coords(arr_shape, coordinates):
    x, y = coordinates
    return (0 <= x < arr_shape[0]) & (0 <= y < arr_shape[1])

In [ ]:
def parse(text: str):
    arr = np.array([[s for s in line] for line in text.strip().split("\n")])
    walls = [(int(x), int(y)) for x, y in zip(*np.where(arr == "#"))]
    start = tuple(map(int, np.where(arr == "S")))
    end = tuple(map(int, np.where(arr == "E")))
    return arr, walls, start, end

In [ ]:
def check_path(mem_shape, start, end, walls):
    queue = []
    heapq.heappush(queue, (0, start))
    visited = []
    dps = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    steps = 0
    iter = 0
    while queue:
        iter += 1
        cost, start_pos = heapq.heappop(queue)
        # print(cost, start_pos)
        if start_pos in visited:
            continue
        visited.append(start_pos)
        
        for pos in [add_tuple(start_pos, dp) for dp in dps]:
            if not check_coords(mem_shape, pos):
                continue
            if pos in walls:
                continue
            if pos in visited:
                continue
            if pos == end:
                steps = cost + 1
                break
            heapq.heappush(queue, (cost + 1, pos))
    return steps, visited


In [ ]:
def get_tile_scores(mem_shape, start, walls):
    queue = []
    out_map = np.full(mem_shape, -1)
    out_map[start] = 0
    heapq.heappush(queue, (0, start))
    visited = []
    dps = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    iter = 0
    while queue:
        iter += 1
        cost, start_pos = heapq.heappop(queue)
        if start_pos in visited:
            continue
        visited.append(start_pos)
        
        for pos in [add_tuple(start_pos, dp) for dp in dps]:
            if not check_coords(mem_shape, pos):
                continue
            if pos in walls:
                continue
            if pos in visited:
                continue
            out_map[pos] = cost + 1

            heapq.heappush(queue, (cost + 1, pos))
    return out_map

In [ ]:
arr, walls, start, end = parse(data_text)
score_map_start = get_tile_scores(arr.shape, start, walls)
score_map_end = get_tile_scores(arr.shape, end, walls)
score_map_print = np.where(score_map_end == -1, arr, score_map_end)


In [ ]:
def iterate_map(map: np.ndarray):
    walls = [(int(x), int(y)) for x, y in zip(*np.where(map == -1))]
    dps = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    res_map = map.copy()
    for wall in walls:
        for dp in dps:
            candidate = add_tuple(wall, dp)
            if not check_coords(map.shape, candidate):
                continue
            if map[candidate] == -1:
                continue
            if res_map[wall] == -1:
                res_map[wall] = map[candidate] + 1
            if map[candidate]  + 1 < res_map[wall]:
                res_map[wall] = map[candidate] + 1
    return res_map

In [ ]:
def get_cheat_ends(start_arr, end_arr, pos, walls, score_th):
    points = [pos]
    scores = [start_arr[pos]]
    visited = [pos]
    targets = []
    first_run = True
    for _ in range(20):
        new_points = []
        new_scores = []
        for i, pos_i in enumerate(points):
            visited.append(pos_i)
            if pos_i not in walls and not first_run:
                if end_arr[pos_i] + scores[i] < score_th and pos_i not in targets:
                    targets.append(pos_i)
                continue
            first_run = False
            dps = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            for dp in dps:
                candidate = add_tuple(pos_i, dp)
                if candidate in visited:
                    continue
                if not check_coords(start_arr.shape, candidate):
                    continue
                new_points.append(candidate)
                new_scores.append(scores[i] + 1)
    
        points = new_points.copy()
        scores = new_scores.copy()
    return len(targets)
                
                

In [ ]:
def get_cheat_ends(start_arr, end_arr, pos, walls, score_th):
    targets = 0
    kernel_size = 20
    for x in range(-kernel_size, kernel_size + 1):
        for y in range(-kernel_size, kernel_size + 1): 
            if abs(x) + abs(y) > kernel_size:
                continue
            candidate = add_tuple((x, y), pos)
            if not check_coords(start_arr.shape, candidate):
                continue
            end_score = end_arr[candidate]
            start_score = start_arr[pos]
            
            if end_score == -1 or start_score == -1:
                continue
            
            score = start_arr[pos] + abs(x) + abs(y) + end_score
            if score <= score_th:
                targets += 1
    return targets

### Part one

In [ ]:
arr, walls, start, end = parse(data_text)
score_map_start = get_tile_scores(arr.shape, start, walls)
score_map_end = get_tile_scores(arr.shape, end, walls)
basic_score, visited = check_path(arr.shape, start, end, walls)
walls_idxs = []
for wall in walls:
    dps = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    min_end_dist = float('inf')
    min_start_dist = float('inf')
    for dp in dps:
        candidate = add_tuple(wall, dp)
        if not check_coords(arr.shape, candidate):
            continue
        end_dist = score_map_end[candidate]
        start_dist = score_map_start[candidate]
        if end_dist != -1:
            min_end_dist = min(min_end_dist, end_dist)
        if start_dist != -1:
            min_start_dist = min(min_start_dist, start_dist)
    saved_time = float(basic_score - (min_start_dist + min_end_dist + 2))
    if saved_time >= 100:
        walls_idxs.append((wall, saved_time))
print(len(walls_idxs))

### Part two

In [ ]:
arr, walls, start, end = parse(data_text)
score_map_start = get_tile_scores(arr.shape, start, walls)
score_map_end = get_tile_scores(arr.shape, end, walls)
basic_score, visited = check_path(arr.shape, start, end, walls)
walls_idxs = []

not_walls = [(int(x), int(y)) for x, y in zip(*np.where(arr != "#"))]

target_cnt = 0
for i, candidate in enumerate(not_walls):
    if candidate not in visited:
        continue
    target_cnt += get_cheat_ends(score_map_start, score_map_end, candidate, walls, basic_score - 100)
print(target_cnt)
